# Projeto Titanic by Barbara Gomes - github @babitta 

## Aumento de gradiente

#### Importando as bibliotecas

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier


#### Carregando os datasets 
Apos salvar os arquivos baixados do kaglle na pasta do projeto utilizaremos a função .read_csv para carregaras bases de dados.
Nomeamos as bases como train e teste.

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


Para visualizar os dados vamos usar a função .sample() que permitira verificar 5 linhas aleatorias da base de dados.

In [7]:
train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
722,723,0,2,"Gillespie, Mr. William Henry",male,34.0,0,0,12233,13.0000,NaN,S
367,368,1,3,"Moussa, Mrs. (Mantoura Boulos)",female,NaN,0,0,2626,7.2292,NaN,C
335,336,0,3,"Denkoff, Mr. Mitto",male,NaN,0,0,349225,7.8958,NaN,S
658,659,0,2,"Eitemiller, Mr. George Floyd",male,23.0,0,0,29751,13.0000,NaN,S
480,481,0,3,"Goodwin, Master. Harold Victor",male,9.0,5,2,CA 2144,46.9000,NaN,S


In [8]:
test.sample(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
119,1011,2,"Chapman, Mrs. John Henry (Sara Elizabeth Lawry)",female,29.0,1,0,SC/AH 29037,26.0000,NaN,S
28,920,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
80,972,3,"Boulos, Master. Akar",male,6.0,1,1,2678,15.2458,NaN,C
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S


#### Seleção dos dados

Apos analise visual dos dados acima vamos retirar as colunas que consideramos desnessarias ao modelo e que não farão diferença na analise dos dados.
Neste caso avaliei as colunas de forma racional pensando o que poderia ou não influenciar no modelo,  mas é claro que dependo da analise é necessaria a avaliação de outros fatores como a correlação entre as variaveis.
Sendo assim as colunas a eliminar seraão: name, cabin e ticket.
usaremos a função .drop()
IMPORTANTE: a partir deste momento tudo que fizermos na base train deve ser replicado na base test , para que os resultados possam ser comparados ao final e comprovar a viabilidade do modelo.

In [9]:
train.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [10]:
#Importante verificar a base novamente para ver como ficou a apos a modificação. 
train.sample(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
402,403,0,3,female,21.0,1,0,9.8250,S
872,873,0,1,male,33.0,0,0,5.0000,S
186,187,1,3,female,NaN,1,0,15.5000,Q
425,426,0,3,male,NaN,0,0,7.2500,S
261,262,1,3,male,3.0,4,2,31.3875,S


####  Tratamento de dados:
##### Valores Texto
Como nosso modelo de Machine Learning só trabalhara com  variaveis numericas, sera necessario tratar os dados relevantes
que estão como texto, as colunas Embarked e Sex.
Para isso vamos utilizar a função .get_dummies(), o objetivo desta função é codificar variáveis, criando novas colunas 
com valores numericos no dataset.
desta forma vamos criar tambem novos datasets para armazenar estas novas informações,vamos nomea-los como 
novo_train e novo_test.

In [11]:
novo_train = pd.get_dummies (train)
novo_test = pd.get_dummies (test)

In [12]:
#Verificar novamente como ficou o dataset apos as modificaçoes.
novo_train.sample(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
68,69,1,3,17.0,4,2,7.9250,1,0,0,0,1
731,732,0,3,11.0,0,0,18.7875,0,1,1,0,0
458,459,1,2,50.0,0,0,10.5000,1,0,0,0,1
265,266,0,2,36.0,0,0,10.5000,0,1,0,0,1
128,129,1,3,NaN,1,1,22.3583,1,0,1,0,0


In [13]:
#Verificar novamente como ficou o dataset apos as modificaçoes.
novo_test.sample(5)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
208,1100,1,33.0,0,0,27.7208,1,0,1,0,0
282,1174,3,NaN,0,0,7.7500,1,0,0,1,0
13,905,2,63.0,1,0,26.0000,0,1,0,0,1
345,1237,3,16.0,0,0,7.6500,1,0,0,0,1
346,1238,2,26.0,0,0,13.0000,0,1,0,0,1


##### Valores Nulos

Da mesma forma que o modelo não aceita valores no formato texto o modelo tambem não aceita valores nulos e portando o devemos trata-los.
DETALHE IMPORTANTE: Como temos 2 bases esta etapa tem q ser realizada separadamente pois teremos tratativas diferentes a depender de quais campos estao nulos nas duas bases.

Vamos começar pela base train.

A primeira parte é identificar se existem valores nulos e onde estão localizados ( em quais colunas), vamos utilizar a função isnull() e a função sum() para quantificar os valores.

In [14]:
novo_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
Sex_female       0
Sex_male         0
Embarked_C       0
Embarked_Q       0
Embarked_S       0
dtype: int64

Como verificamos acima temos 177 valores nulos na coluna AGE (idade), sendo assim vamos substituir os nulos pela média de idade dos ocupantes, lembrando que cada coluna com valores nulos pede uma tratativa diferente devido ao tipo de dado e situação.

Utilizamos a formula fillna() para subistituição dos valores nulos e mean() para média 

In [15]:
novo_train["Age"].fillna(novo_train["Age"].mean(),inplace=True)


In [16]:
#vamos testar novamente para ver se deu certo 
novo_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

Agora vamos ao dataset test
identificar se existem valores nulos e onde estão localizados 

In [17]:
novo_test.isnull().sum()

PassengerId     0
Pclass          0
Age            86
SibSp           0
Parch           0
Fare            1
Sex_female      0
Sex_male        0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

Observamos que na base test temos 86 valores nulos na coluna AGE e 1 valor nulo na coluna FARE, desta forma vamos replicar a formula para tratar a oluna age primeiro.

In [18]:
novo_test["Age"].fillna(novo_test["Age"].mean(),inplace=True)

In [19]:
#vamos testar  para ver se deu certo 
novo_test.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           1
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

Agora vamos tratar a coluna FARE, como esta coluna se trata de preço da passagem vamos imputar a média igual fizemos com a idade dos passageiros.

In [20]:
novo_test["Fare"].fillna(novo_test["Fare"].mean(),inplace=True)

In [21]:
#testar para ver se deu certo 
novo_test.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

Base de dados tratada , vamos criar o modelo.

#### Criando o Modelo de Machine Learning - aumento de gradiente
Nosso objetivo neste caso é prever a sobrevivencia dos passageiros, sendo assim vamos definir as nossas variaveis como:

Feature - dados de aprendizagem (passageiros), 

Target - o que queremos medir (sobrevivencia)


In [22]:
Feature = novo_train.drop('Survived',axis =1)
Target = novo_train['Survived']

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
gbk = GradientBoostingClassifier()
gbk.fit(Feature, Target)


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [28]:
gbk.score (Feature,Target)

0.8978675645342312

In [30]:
submission = pd.DataFrame()
submission['PassengerId'] = novo_test['PassengerId']
submission['Survived'] = gbk.predict(novo_test)

In [31]:
submission.to_csv('submission_GBK.csv', index=False)